In [ ]:
from rlcube.models.models import DNN
from rlcube.envs.cube2 import Cube2Env
import torch

net = DNN()
net.load("checkpoints/checkpoint_final.pth")
net.eval()

DNN(
  (fc_in): Linear(in_features=144, out_features=512, bias=True)
  (residual_blocks): ModuleList(
    (0-3): 4 x ResidualBlock(
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=512, out_features=1024, bias=True)
      (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (fc2): Linear(in_features=1024, out_features=512, bias=True)
    )
  )
  (fc_value): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
  (fc_policy): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=12, bias=True)
  )
)

In [103]:
import numpy as np
from rlcube.models.search import MonteCarloTree

env = Cube2Env()

actions = []
obs = []
for _ in range(10):
    action = env.action_space.sample()
    actions.append(action.item())
    env.step(action)
    obs.append(env.obs())

obs = torch.tensor(np.array(obs), dtype=torch.float32)
values, policies = net(obs)
print(actions)
print(values)


tree = MonteCarloTree(env.obs(), max_simulations=1000)
if tree.is_solved:
    print([action for _, action in tree.solved_path])

[7, 11, 6, 7, 7, 10, 1, 0, 3, 3]
tensor([[ 0.9634],
        [-0.0930],
        [-0.8327],
        [-0.0930],
        [-0.8955],
        [-1.8250],
        [-4.0525],
        [-1.8250],
        [-3.0264],
        [-3.6782]], grad_fn=<AddmmBackward0>)


  1%|          | 8/1000 [00:00<00:10, 99.11it/s]

[0, 2, 5, 2, 8, 6]
